In [1]:
from cosapp.base import System
from components.pipe import Pipe
from components.reservoir import IntakeReservoir, DischargeReservoir
from components.pump import Pump
from components.fluid import Fluid
from cosapp.drivers import NonLinearSolver

In [2]:
# generate piping

class Circuit(System):
    def setup(self):
        # setting systems
        self.add_child(IntakeReservoir("res_in"), pulling = ["density", "gravity", "atmosphere"])
        self.add_child(Pipe("pipe_1"), pulling=["kin_viscosity", "density", "mass_flow"])
        self.add_child(Pump("pump"), pulling = ["density", "mass_flow"])
        self.add_child(Pipe("pipe_2"), pulling=["kin_viscosity", "density", "mass_flow"])
        self.add_child(DischargeReservoir("res_out"), pulling = ["density", "gravity", "atmosphere"])
        
        # connecting
        self.connect(self.res_in.outwards, self.pipe_1.inwards, {"pressure_out": "pressure_in"})
        self.connect(self.pipe_1.outwards, self.pump.inwards, {"pressure_out": "pressure_in"})
        self.connect(self.pump.outwards, self.pipe_2.inwards, {"pressure_out": "pressure_in"})
        self.connect(self.pipe_2.outwards, self.res_out.inwards, {"pressure_out": "pressure_in"})

        self.add_unknown("mass_flow")
        self.add_driver(NonLinearSolver("circuit_solver"))
    
    def compute(self):
        pass        

In [10]:
# give a fluid to the circuit

class HydroCircuit(System):
    def setup(self):
        self.add_child(Circuit('piping'))
        self.add_child(Fluid('water'))
        
        self.connect(self.piping.inwards, self.water.outwards, ['density', 'kin_viscosity'])
    
    def compute(self):
        pass

# instantiate
s = HydroCircuit('system')

# set fluid properties
s.water.density = 1e3
s.water.kin_viscosity = 1e-6

# set piping custom properties
s.piping.res_in.level = 10
s.piping.pipe_2.diameter = 0.5

# run
s.run_drivers()

# plot some results to check
print(s.piping.mass_flow)
print(s.piping.pipe_1.fluid.reynolds)
print(s.piping.pipe_2.fluid.reynolds)

1.0
22731.005144925643
28942014.642097767
57884029.284195535
